# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>


This project wants to show if it is convenient to open a **Coffee shop** in **Tirana**, Albania. People who might be interested in this analysis are future young entrepreneurs, who are new to the Coffee shop industry.

Tirana has a population density of 500 people per square kilometre, is the most densely populated municipality in the country. About 1/4 of the country's total population is located in Tirana.
Now, since the citizens of Tirana like a good coffee in company, it may be a good idea to open a Coffee shop, let find it out, if it really is so!

We will try to detect **locations that are not already crowded with Coffee shops**. We are interested in locations with **no Coffee shops in nearby** and for sure **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>


Based on definition of our problem, factors that will influence our decission are:
* number of existing Coffe shops in the neighborhood
* distance to Coffe shops in the neighborhood, if any:
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of Coffe shops and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Tirana city center will be obtained using **Google Maps API geocoding** of the known Skanderbeg Square.


##### TIRANA CITY CENTER

Let's first find the latitude & longitude of Tirana city center, using specific, well known address and Google Maps geocoding API.

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import sys
from bs4 import BeautifulSoup
import os
from geopy.geocoders import Nominatim 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline


print('Libraries imported.')

Libraries imported.


In [3]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 98 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 3.4 MB/s  eta 0:00:01


In [9]:
import json
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='UR0UP5N0SGHDMNDAKQHZG213Y1ESEGPFOZ2OZHKSBRZQQ4ZM',
client_secret='GDFCMZSWXMJZUOAEX3ERJVLHHQQFIL4ZPTINJIWWJZCVLVYP',
v='20201119',
ll='41.328868,19.818570',
query='coffee',
limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
print (data)

{'meta': {'code': 200, 'requestId': '5fd775bab8aa6d5abd7f0f73'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'warning': {'text': 'There aren\'t a lot of results for "coffee." Try something more general, reset your filters, or expand the search area.'}, 'suggestedRadius': 1619, 'headerLocation': 'Tirana', 'headerFullLocation': 'Tirana', 'headerLocationGranularity': 'city', 'query': 'coffee', 'totalResults': 72, 'suggestedBounds': {'ne': {'lat': 41.33217536675265, 'lng': 19.81920277856529}, 'sw': {'lat': 41.3294757069672, 'lng': 19.822262660200614}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '562a5fef498e8e32d797c9c2', 'name': 'Mulliri Vjeter', 'location': {'address': 'Rr. Barrikadave', 'lat': 41.330825536859926, 'lng': 19.82073271938295, 

In [11]:
CLIENT_ID = 'UR0UP5N0SGHDMNDAKQHZG213Y1ESEGPFOZ2OZHKSBRZQQ4ZM' # your Foursquare ID
CLIENT_SECRET = 'GDFCMZSWXMJZUOAEX3ERJVLHHQQFIL4ZPTINJIWWJZCVLVYP' # your Foursquare Secret
VERSION = '20201119' # Foursquare API version
LIMIT = 15000 # A default Foursquare API limit value
radius = 10000

In [12]:
# Coorinates for Connaught Place abbreviated as CP
CP_latitude = 41.328868
CP_longitude = 19.818570

In [13]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CP_latitude, 
    CP_longitude, 
    radius, 
    LIMIT,
)

In [14]:
results = requests.get(url).json()
results.values();
results

{'meta': {'code': 200, 'requestId': '5fd7764641131e59a637f703'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Tirana',
  'headerFullLocation': 'Tirana',
  'headerLocationGranularity': 'city',
  'totalResults': 88,
  'suggestedBounds': {'ne': {'lat': 41.41886809000009,
    'lng': 19.938197488412005},
   'sw': {'lat': 41.23886790999991, 'lng': 19.698942511587997}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5726844a38fa6e0abbf079c4',
       'name': 'The Plaza',
       'location': {'address': 'Rruga 28 Nëntori',
        'lat': 41.327662102745755,
        'lng': 19.82158864976927,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.327662102745755,
          'ln

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
venues_neighborhood = [results['response']['groups'][0]['items'][n]['venue']['location']['formattedAddress'] for n in range(len(venues))]  
venues_distance = [results['response']['groups'][0]['items'][n]['venue']['location']['distance'] for n in range(len(venues))]  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

[venues_neighborhood[k][0] for k in range(len(venues_neighborhood))]

#nearby_venues['neighborhood'] = venues_neighborhood[0][0]
nearby_venues['neighborhood'] =  [venues_neighborhood[k][0] for k in range(len(venues_neighborhood))]
nearby_venues['distance [m]'] = venues_distance

df = nearby_venues
df

NameError: name 'json_normalize' is not defined

## Methodology <a name="data"></a>

## Analysis <a name="data"></a>

## Results and Discussion <a name="data"></a>

## Conclusion <a name="data"></a>